# Trying to make a dl model from scratch to classify images

In [36]:
import numpy as np
import os
import pandas as pd
from PIL import Image
import io

# improting data

In [126]:
# importing db from parquet files
train_data_path = os.path.join('mnist', 'train-00000-of-00001.parquet')
test_data_path = os.path.join('mnist', 'test-00000-of-00001.parquet')

In [127]:
train_df = pd.read_parquet(train_data_path)
test_df = pd.read_parquet(test_data_path)

In [128]:
train_df.head()

,image,label
0,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,5
1,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,0
2,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,4
3,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",1
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,9


In [129]:
test_df.head()

,image,label
0,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",7
1,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",2
2,"{'bytes': b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...",1
3,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,0
4,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,4


In [130]:
# splitting labels from db
y_train = train_df.pop('label').values
y_test = test_df.pop('label').values

In [131]:
X_train = train_df.values
X_test = test_df.values

In [132]:
def preprocess_image(image_bytes):
    # Load image from bytes
    image = Image.open(io.BytesIO(image_bytes)).convert("L")  # Convert to grayscale

    # Convert image to NumPy array
    image_array = np.array(image, dtype=np.float32)

    # Normalize pixel values (0-255 → 0-1)
    image_array /= 255.0

    # Flatten the image to match MLP input shape (1, 784)
    image_flattened = image_array.reshape(1, 28*28)

    return image_flattened

## sample image

In [133]:
image = Image.open(io.BytesIO(X_train[0][0]['bytes']))
image

In [134]:
image.size

(28, 28)

## flatten the image to a 784 single line vector space for inputs or activations for the input layer of our model

In [135]:
prepro = preprocess_image(X_train[0][0]['bytes'])
prepro.size

784

In [136]:
# pre process all images for train data
X_train = np.array([preprocess_image(image[0]['bytes']) for image in X_train])
X_train = X_train.reshape(X_train.shape[0], 784)

In [137]:
# pre process all images for test data
X_test = np.array([preprocess_image(image[0]['bytes']) for image in X_test])
X_test = X_test.reshape(X_test.shape[0], 784)

In [138]:
# working of np.eye
np.eye(10)[[0, 1, 5]] # so it gives you this line vector kinda with 1 at the neuron which we wanna activate

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [139]:
y_train

array([5, 0, 4, ..., 5, 6, 8], shape=(60000,))

## one hot encode to match the output space of the model

In [140]:
def one_hot_encode(y, num_classes=10):
    return np.eye(num_classes)[y]

y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)

# functions defined somehow from gpt

In [141]:
# ReLU Activation
def relu(Z):
    return np.maximum(0, Z)

# ReLU Derivative
def relu_derivative(Z):
    return (Z > 0).astype(float)

# Softmax Activation
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # Stability trick
    return expZ / np.sum(expZ, axis=1, keepdims=True)


## initial network params

In [ ]:
# Define network architecture
input_size = 784
hidden_size = 128
output_size = 10
learning_rate = 0.01

# Initialize weights and biases
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

## why multiply by 0.01

## 1 Prevents Large Activations & Exploding Gradients
If weights are too large, activations in the network will also be large.
This can cause exploding gradients, where values grow too big during backpropagation, making training unstable.
By initializing weights close to zero, activations remain small and controlled.
## 2 Encourages Efficient Learning
If weights are too large, neurons can become saturated (for activation functions like sigmoid or tanh), leading to vanishing gradients.
Small weights ensure gradients remain in a good range for optimization.

# More function definitions from gpt

In [143]:
def forward_propagation(X):
    global W1, b1, W2, b2
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2


In [144]:
def compute_loss(Y_true, Y_pred):
    m = Y_true.shape[0]
    return -np.sum(Y_true * np.log(Y_pred + 1e-8)) / m  # Avoid log(0)


In [ ]:
def backpropagation(X, Y_true, Z1, A1, Z2, A2):
    global W1, b1, W2, b2, learning_rate
    
    m = X.shape[0]
    # using gradient descent here. 
    # Output layer error
    dZ2 = A2 - Y_true
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Hidden layer error
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2




# Training

In [ ]:
from sklearn.utils import shuffle

epochs = 20
batch_size = 32

for epoch in range(epochs):
    X_train, y_train = shuffle(X_train, y_train)  # Shuffle for randomness

    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        Y_batch = y_train[i:i+batch_size]

        # Forward pass
        Z1, A1, Z2, A2 = forward_propagation(X_batch)

        # Compute loss
        loss = compute_loss(Y_batch, A2)
        
        # Backward pass
        backpropagation(X_batch, Y_batch, Z1, A1, Z2, A2)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")


Epoch 1/20, Loss: 0.0412
Epoch 2/20, Loss: 0.0436
Epoch 3/20, Loss: 0.2108
Epoch 4/20, Loss: 0.0646
Epoch 5/20, Loss: 0.0563
Epoch 6/20, Loss: 0.1042
Epoch 7/20, Loss: 0.0270
Epoch 8/20, Loss: 0.0624
Epoch 9/20, Loss: 0.0864
Epoch 10/20, Loss: 0.0280
Epoch 11/20, Loss: 0.2330
Epoch 12/20, Loss: 0.1360
Epoch 13/20, Loss: 0.0851
Epoch 14/20, Loss: 0.3090
Epoch 15/20, Loss: 0.0717
Epoch 16/20, Loss: 0.0494
Epoch 17/20, Loss: 0.1005
Epoch 18/20, Loss: 0.0590
Epoch 19/20, Loss: 0.0192
Epoch 20/20, Loss: 0.0354


# Evaluation

In [149]:
def predict(X):
    _, _, _, A2 = forward_propagation(X)
    return np.argmax(A2, axis=1)

y_pred = predict(X_test)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(y_pred == y_true)

print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 97.28%


# improvement suggestions from gpt

Adding More Hidden Neurons – Try increasing the number of neurons in the hidden layer.

Adding Another Hidden Layer – A deeper network might capture more patterns.

Using Batch Normalization – Helps stabilize training and improves generalization.

Optimizing Learning Rate – Try a learning rate scheduler or Adam optimizer instead of plain SGD.